In [72]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import datetime
import re
import sqlalchemy as sa
from sqlalchemy import create_engine
import pymysql

In [73]:
#creating connection with MySQL datbase

user = "brenobertolucci"
password = "Panda190322"
url_banco = "brenobertolucci.mysql.pythonanywhere-services.com"
nome_db = "brenobertolucci$dados_anbima"
conn_str = f"mysql+pymysql://{user}:{password}@{url_banco}/{nome_db}"
engine = create_engine(conn_str)


In [19]:
#Selecting day of extraction
yesterday =  datetime.date.today() - datetime.timedelta(days=1)

year_ext = str(yesterday.year)[2:]

if len(str(yesterday.month)) == 1:
    month_ext = "0"+str(yesterday.month)
else:
    month_ext = str(yesterday.month)

    
if len(str(yesterday.day)) == 1:
    day_ext = "0"+str(yesterday.day)
else:
    day_ext = str(yesterday.day)


In [20]:
def clean_cupom(index):
    
    if "+" in index:
        clean_cupom= index.split('+')[1].replace("%","").strip()
    else:
        clean_cupom = index.split('do')[0].replace("%","").strip()
        
    return clean_cupom

In [21]:
def clean_index(index):
    
    if "+" in index:
        clean_index = index.split('+')[0].strip()
    else:
        clean_index = "%DI"
        
    return clean_index

In [22]:
def clean_name(name):
    
    pattern_dirty = r'[\(][\*]*[\)]'
    pattern_clean = r''
    name_clean = re.sub(pattern_dirty,pattern_clean,name).strip().replace("S/A","SA").replace("S.A.","SA").replace("S.A","SA")
    
    return name_clean

In [23]:
def clean_string(string_to_clean):
    
    
    pattern_dirty = r'[,]'
    pattern_clean = r'.'
    string_clean = re.sub(pattern_dirty,pattern_clean,string_to_clean)
    
    return string_clean
    

In [68]:
def read_sql(query, engine):
    with engine.begin() as conn:
        results = pd.read_sql_query(sa.text(query), conn)
    return results
    
def execute_sql(query, engine):
    with engine.begin() as conn:
        results = conn.execute(
            sa.text(query)
            )
    return results

def df_to_sql(df, table, engine):
    with engine.begin() as conn:
        df.to_sql(table ,conn, index = False, index_label = False, if_exists = "append") 

In [25]:
#extracting info of debentures

url_debs  = f"https://www.anbima.com.br/informacoes/merc-sec-debentures/arqs/db{year_ext}{month_ext}{day_ext}.txt"
df_debs = pd.read_csv(url_debs,sep="@",encoding="latin1",skiprows=1)

#cleaning index

df_debs['Indexador'] = df_debs['Índice/ Correção'].map(clean_index)


#cleaning cupom

df_debs['Cupom'] = df_debs['Índice/ Correção'].map(clean_cupom).map(clean_string).astype(float)


#converting the columns "to date

df_debs['Referência NTN-B'] = pd.to_datetime(df_debs['Referência NTN-B'])
df_debs['Referência NTN-B'] = df_debs['Referência NTN-B'].dt.strftime('%Y-%m-%d')
df_debs['Repac./  Venc.'] = pd.to_datetime(df_debs['Repac./  Venc.'])

#cleaing name

df_debs['Nome'] = df_debs['Nome'].map(clean_name)

#converting  columns to float

df_debs['PU'] = df_debs['PU'].replace("N/D",str("1000000")).map(clean_string).replace("1000000",np.nan).astype(float) 
df_debs['Duration'] = df_debs['Duration'].replace("N/D",str("1000000")).map(clean_string).replace("1000000",np.nan).astype(float) 
df_debs['% PU Par'] = df_debs['% PU Par'].replace("N/D",str("1000000")).map(clean_string).replace("1000000",np.nan).astype(float) 
df_debs['Taxa de Compra'] = df_debs['Taxa de Compra'].replace("--",str("1000000")).map(clean_string).replace("1000000",np.nan).astype(float) 
df_debs['Taxa Indicativa'] = df_debs['Taxa Indicativa'].replace("--",str("1000000")).map(clean_string).replace("1000000",np.nan).astype(float) 
df_debs['Taxa de Venda'] = df_debs['Taxa de Venda'].replace("--",str("1000000")).map(clean_string).replace("1000000",np.nan).astype(float) 


#creating column date

df_debs['Data'] = yesterday

In [26]:
#extracting info of public titles
url_pt  = f"https://www.anbima.com.br/informacoes/merc-sec/arqs/ms{year_ext}{month_ext}{day_ext}.txt"
df_pt = pd.read_csv(url_pt,sep="@",encoding="latin1",skiprows=1)

#converting the columns "Data Vencimento" to date

df_pt['Data Vencimento'] = df_pt['Data Vencimento'].astype(str)
df_pt['Data Vencimento'] = pd.to_datetime(df_pt['Data Vencimento'])
df_pt['Data Vencimento'] = df_pt['Data Vencimento'].dt.strftime('%Y-%m-%d')

#converting rates columns to float

df_pt['Tx. Compra'] = df_pt['Tx. Compra'].map(clean_string).astype(float) 
df_pt['Tx. Indicativas'] = df_pt['Tx. Indicativas'].map(clean_string).astype(float) 
df_pt['Tx. Venda'] = df_pt['Tx. Venda'].map(clean_string).astype(float) 


#selecting only relevants columns
df_pt = df_pt[['Titulo','Data Vencimento','Tx. Compra','Tx. Indicativas','Tx. Venda']]

In [27]:
#adding the NTN-B rate to the Debs DF
df_full = pd.merge(df_debs, df_pt, left_on="Referência NTN-B", right_on = "Data Vencimento", how="left")

#creating key
df_full['ID'] = df_full["Código"] + str(yesterday)

#cleaning nans
df_full['Tx. Compra'] = df_full['Tx. Compra'].replace(np.nan,0)
df_full['Tx. Indicativas'] = df_full['Tx. Indicativas'].replace(np.nan,0)
df_full['Tx. Venda'] = df_full['Tx. Venda'].replace(np.nan,0)

#calculating spreads

df_full["Spread_Compra"] = ((1+df_full['Taxa de Compra']/100)/(1+df_full["Tx. Compra"]/100)-1)*100
df_full["Spread_Indicativo"] = ((1+df_full['Taxa Indicativa']/100)/(1+df_full["Tx. Indicativas"]/100)-1)*100
df_full["Spread_Venda"] = ((1+df_full['Taxa de Venda']/100)/(1+df_full["Tx. Venda"]/100)-1)*100


#selecting only relevants columns and chanhing names


df_full=df_full[['ID','Data','Código',"Nome","Repac./  Venc.","Indexador",'Cupom',"Taxa de Compra",
          "Taxa Indicativa","Taxa de Venda",'PU','% PU Par','Duration',
          'Referência NTN-B','Spread_Compra','Spread_Indicativo','Spread_Venda']]

df_full.columns = ['ID','Data','Ticker','Emissor',"Vencimento","Indexador",'Cupom',"Taxa_Compra",
          "Taxa_Indicativa","Taxa_Venda",'PU','Perc_Par','Duration',
          'Ref_B','Spread_Compra','Spread_Indicativo','Spread_Venda']



In [28]:
#exporting daily files:

#sql

df_to_sql(df_full,"dados_anbima",engine)

#excel

name_file_xlsx = str(year_ext)+str(month_ext)+str(day_ext)+"_ANBIMA.xlsx"
df_full.to_excel(f'daily_files/{name_file_xlsx}')

#json
name_file_json = str(year_ext)+str(month_ext)+str(day_ext)+"_ANBIMA.json"
df_full.to_json(f'daily_files/{name_file_json}', orient = 'index')



In [74]:
#creating table with sectos 

#importing historic table

df_historico = read_sql('select * from dados_anbima',engine)

#importing setores

setores = read_sql('select * from setores_anbima',engine)

#merging tables

df_historico_setores = pd.merge(df_historico, setores,how="left")

#exporting files
df_historico_setores.to_excel('daily_files/historico_setores.xlsx')
df_historico_setores.to_json('daily_files/historico_setores.json', orient = 'index')
